In [18]:
from datascience import *
import numpy as np
# Your analysis and answers here:
patients = Table.read_table('breast-cancer.csv').drop('ID')
patients

Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
5,1,1,1,2,1,3,1,1,0
5,4,4,5,7,10,3,2,1,0
3,1,1,1,2,2,3,1,1,0
6,8,8,1,3,4,3,7,1,0
4,1,1,3,2,1,3,1,1,0
8,10,10,8,7,10,9,7,1,1
1,1,1,1,2,10,3,1,1,0
2,1,2,1,2,1,3,1,1,0
2,1,1,1,2,1,1,1,5,0
4,2,1,1,2,1,2,1,1,0


In [19]:
def prob(data, cancer, fieldType, fieldValue): 
    cancerTable = patients.where('Class', cancer)
    matches = cancerTable.where(fieldType, fieldValue).num_rows
    if cancerTable.num_rows==0:
        return 0
    return matches / cancerTable.num_rows

def naive_bayes(data, blandChromatin, epithelialSize, cancer):
    prior = data.where('Class', cancer).num_rows / patients.num_rows
    probability = prior
    probability *= prob(data, cancer, 'Bland Chromatin', blandChromatin)
    probability *= prob(data, cancer, 'Single Epithelial Cell Size', epithelialSize)
    return probability

In [20]:
def calcFieldProb(trainTable):
    for i in range(2):
        def calcProb(blandChromatin, epithelialSize):
            return naive_bayes(patients, blandChromatin, epithelialSize, i)
        probColumn = patients.apply(calcProb, 'Bland Chromatin', 'Single Epithelial Cell Size')
        trainTable = trainTable.with_column("Cancer = " + str(i), probColumn)
    return trainTable

calcFieldProb(patients)

Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class,Cancer = 0,Cancer = 1
5,1,1,1,2,1,3,1,1,0,0.14633,0.0046313
5,4,4,5,7,10,3,2,1,0,0.000824397,0.00198484
3,1,1,1,2,2,3,1,1,0,0.14633,0.0046313
6,8,8,1,3,4,3,7,1,0,0.0115416,0.00948314
4,1,1,3,2,1,3,1,1,0,0.14633,0.0046313
8,10,10,8,7,10,9,7,1,1,0,0.00060648
1,1,1,1,2,10,3,1,1,0,0.14633,0.0046313
2,1,2,1,2,1,3,1,1,0,0.14633,0.0046313
2,1,1,1,2,1,1,1,5,0,0.173255,0.000257295
4,2,1,1,2,1,2,1,1,0,0.179108,0.000900531


In [21]:
def chooosePrediction(column0, column1):
    return np.argmax([column0, column1])
    
def classifer(trainTable):
    trainTable = calcFieldProb(trainTable)
    return trainTable.with_column('Prediction', trainTable.apply(chooosePrediction, 'Cancer = 0', 'Cancer = 1'))

classifer(patients)

Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class,Cancer = 0,Cancer = 1,Prediction
5,1,1,1,2,1,3,1,1,0,0.14633,0.0046313,0
5,4,4,5,7,10,3,2,1,0,0.000824397,0.00198484,1
3,1,1,1,2,2,3,1,1,0,0.14633,0.0046313,0
6,8,8,1,3,4,3,7,1,0,0.0115416,0.00948314,0
4,1,1,3,2,1,3,1,1,0,0.14633,0.0046313,0
8,10,10,8,7,10,9,7,1,1,0,0.00060648,1
1,1,1,1,2,10,3,1,1,0,0.14633,0.0046313,0
2,1,2,1,2,1,3,1,1,0,0.14633,0.0046313,0
2,1,1,1,2,1,1,1,5,0,0.173255,0.000257295,0
4,2,1,1,2,1,2,1,1,0,0.179108,0.000900531,0


In [22]:
def checkAccuracy(table):
    return sum(table.column("Class") == table.column("Prediction")) / table.num_rows

checkAccuracy(classifer(patients))

0.92972181551976574